In [1]:
import numpy as np
from random import randint

# Задание 1

Процесс конструирования выравниваний строк длин $ n $ и $ m $ сводится к размещению символов одной строки среди символов другой без изменения порядка следования символов каждой из строк между собой. В данном случае несколько подряд идущих символов одной строки соответствуют разрыву соответствующего размера в другой. Таких комбинаций сохраняющих порядок внутри каждой из строк существует $ C^{n}_{n+m} = C^{m}_{n+m} $

# Задание 2

Можно считать строки из файла

In [ ]:
# a, b = '', ''

# with open('strings_for_alignment.txt', 'r') as f:
#     a = f.readline().rstrip()
#     b = f.readline().rstrip()

А можно ввести вручную

In [285]:
a = 'editing'
b = 'distance'

Задаем массив весов

In [286]:
W = {'match':1, 'mismatch':-1, 'gap':-1}

## Выравниваем 

In [287]:
def diff(i, j):
    if a[i] == b[j]:
        return 'match'
    return 'mismatch'

In [299]:
n, m = len(a), len(b)

D = np.empty((n+1, m+1)) #, dtype=np.int)
D[0,:] = np.arange(m+1)*W['gap']
D[:,0] = np.arange(n+1)*W['gap']

In [300]:
for i in range(1, n+1):
    for j in range(1, m+1):
        D[i, j] = max(D[i-1, j] + W['gap'], D[i, j-1] + W['gap'], D[i-1, j-1] + W[diff(i-1, j-1)])

Восстанавливаем выравнивания

In [301]:
i, j = n, m
a1, b1 = '', ''

while i + j > 0:
    if D[i, j] == D[i-1, j] + W['gap']:
        a1 += a[i-1]
        b1 += '-'
        i -= 1
    elif D[i, j] == D[i, j-1] + W['gap']:
        a1 += '-'
        b1 += b[j-1]
        j -= 1
    else:
        a1 += a[i-1]
        b1 += b[j-1]
        i -= 1
        j -= 1

In [291]:
print(a1[::-1])
print(b1[::-1])

edi-ting-
-distance


### Тест 1

In [292]:
a = 'qqqaaaaaGaaaaa'
b = 'aaaaaCaaaaabbb'

In [296]:
print(a1[::-1])
print(b1[::-1])

qqqaaaaaGaaaaa---
---aaaaaCaaaaabbb


### Тест 2

In [297]:
W = {'match':1, 'mismatch':-1, 'gap':-0.499}

In [302]:
print(a1[::-1])
print(b1[::-1])

qqqaaaaa-Gaaaaa---
---aaaaaC-aaaaabbb


# Задание 3

Считывем из файла таблицу весов PAM250

In [23]:
W = np.empty((25,25), dtype=np.int)
ind = {}

f = open('PAM_250.txt', 'r')
amino = f.readline().split()[1:]
ind = {amino[i]:i for i in range(25)}
for i in range(25):
    W[i,:] = np.array(f.readline().split()[2:])
    
f.close()

Создаем независимо мутировавшие аминокислотные последовательности 

In [3]:
def mutogenesis(s, num, max_len):
    for _ in range(num):
        pos = randint(0, len(s)-1)
        ins = randint(1,max_len)
        dlt = randint(1,max_len)
        s = s[:pos] + ''.join([amino[randint(0, 19)] for _ in range(ins)]) + s[pos+dlt:]
    return s

In [4]:
A = ''.join([amino[randint(0, 19)] for _ in range(70)])
B = A
A = mutogenesis(A, 10, 3)
B = mutogenesis(B, 10, 3)
print(A)
print(B)

CYVSMNDKILFDMLYHEGRYELDLMTHGVYYTLLWATADLRTMLYDSQNGSFSAQTYDFDNQKMISCEQA
CWDMNDKILFGYHEGGFVMTHYVYYCGLWATAPLWECISAQMCNGSFWLSAQSVYEVFKMISCEQS


Понравились? Запомним их

In [5]:
a, b = A, B

## Выравниваем

In [16]:
def gap_w(s):
    w = np.zeros((len(s)+1))
    for i in range(1, len(s)+1):
        w[i] = w[i-1] + W[ind[s[i-1]], ind['-']]
    return w

In [25]:
n, m = len(a), len(b)

D = np.empty((n+1, m+1), dtype=np.int)
D[:,0] = gap_w(a)
D[0,:] = gap_w(b)

In [26]:
for i in range(1, n+1):
    for j in range(1, m+1):
        a_gap = D[i, j-1] + W[ind[b[j-1]], ind['-']]
        b_gap = D[i-1, j] + W[ind[a[i-1]], ind['-']]
        match = D[i-1, j-1] + W[ind[a[i-1]], ind[b[j-1]]]
        D[i, j] = max(a_gap, b_gap, match)

Восстанавливаем выравнивания

In [9]:
def edit_dist(a, b):
    w = 0
    for i in range(len(a)):
        w += W[ind[a[i]], ind[b[i]]]
    return w

In [27]:
i, j = n, m
a1, b1, m = '', '', ''

while i + j > 0:
    if D[i, j] == D[i-1, j] + W[ind[a[i-1]], ind['-']]:
        a1 += a[i-1]
        b1 += '-'
        m += ' '
        i -= 1
    elif D[i, j] == D[i, j-1] + W[ind[b[j-1]], ind['-']]:
        a1 += '-'
        b1 += b[j-1]
        m += ' '
        j -= 1
    else:
        a1 += a[i-1]
        b1 += b[j-1]
        if a[i-1] == b[j-1]:
            m += '|'
        else:
            m += ' '
        i -= 1
        j -= 1

In [28]:
assert edit_dist(a1,b1) == D[-1,-1]

In [14]:
print(a1[::-1])
print(m[::-1])
print(b1[::-1])

CYVSMNDKILFDMLYHEGRYELDLMTHGVYYTLLWATADLRTMLYDSQ-NGSFS-AQTYDFDNQKMISCEQA
|   |||||||   ||||      ||| |||  ||||| |         ||||           ||||||| 
CW-DMNDKILFG--YHEGGF-V--MTHYVYYCGLWATAPLWECISAQMCNGSFWLSAQSVYEVFKMISCEQS


### Test 

Попробуем испортить как минимум последнюю позицию выравнивания

In [24]:
W[ind[a1[0]], ind[b1[0]]] = -9999
W[ind[b1[0]], ind[a1[0]]] = -9999

In [29]:
print(a1[::-1])
print(m[::-1])
print(b1[::-1])

CYVSMNDKILFDMLYHEGRYELDLMTHGVYYTLLWATADLRTMLYDSQ-NGSFSAQTYD-FDNQKMISCEQ-A
|   |||||||   ||||      ||| |||  ||||| |         ||||           |||||||  
CW-DMNDKILFG--YHEGGF-V--MTHYVYYCGLWATAPLWECISAQMCNGSFWLSAQSVYEVFKMISCEQS-


Готово!